In [1]:
#import os
#os.environ['CUDA_LAUNCH_BLOCKING'] = 1


## Standard libraries
import os
from copy import deepcopy

## Imports for plotting
import matplotlib.pyplot as plt
plt.set_cmap('viridis')
%matplotlib inline
#from IPython.display import set_matplotlib_formats
#set_matplotlib_formats('svg', 'pdf') # For export
import matplotlib
matplotlib.rcParams['lines.linewidth'] = 0 #2.0
#import seaborn as sns
#sns.set()

## tqdm for loading bars
from tqdm.notebook import tqdm

## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim

## Torchvision
import torchvision
from torchvision import transforms
import numpy as np
from PIL import Image

# PyTorch Lightning
import pytorch_lightning as pl
#try:
#    import pytorch_lightning as pl
#except ModuleNotFoundError: # Google Colab does not have PyTorch Lightning installed by default. Hence, we do it here if necessary
#    !pip install --quiet pytorch-lightning>=1.4
#    import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

# Import tensorboard
%load_ext tensorboard

# Path to the folder where the datasets are/should be downloaded (e.g. CIFAR10)
#DATASET_PATH = "../data"
# Path to the folder where the pretrained models are saved
CHECKPOINT_PATH = '/home/azamat.kaibaldiyev/Tasmania_ohara_labels/checkpoints_dems_32/'

# In this notebook, we use data loaders with heavier computational processing. It is recommended to use as many
# workers as possible in a data loader, which corresponds to the number of CPU cores
NUM_WORKERS = os.cpu_count()

# Setting the seed
pl.seed_everything(42)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)
print("Number of workers:", NUM_WORKERS)

[rank: 0] Global seed set to 42


Device: cuda:0
Number of workers: 40


<Figure size 640x480 with 0 Axes>

In [2]:
#with open('files_remove_indices/Ohara_train_set_indexes.npy', 'wb') as f:
#    np.save(f, train_index)

#with open('files_remove_indices/Ohara_test_set_indexes.npy', 'wb') as f:
#    np.save(f, test_index)

with open('files_remove_indices/Ohara_train_set_indexes.npy', 'rb') as f:
    train_set_indexes = np.load(f)

with open('files_remove_indices/Ohara_test_set_indexes.npy', 'rb') as f:
    test_set_indexes = np.load(f)

In [2]:

from torch.utils.data import Dataset, DataLoader
import glob
import rasterio
dataset_path = 'Tasmania_ohara_labels/cropped_dems_32/'
dataset_path_with_labels = 'Tasmania_ohara_labels/cropped_dems_32_with_labels/'

class SwedishDataset(Dataset):
    def __init__(self,  transform  = None, dataset_path = None, remove_list = None):
        self.imgs_path = dataset_path
        self.file_list_dem = sorted(glob.glob(self.imgs_path + "*"))

        self.transform = transform
        
    def __len__(self):
        return len(self.file_list_dem)

          
    def __getitem__(self, idx):
        img_path_dem = self.file_list_dem[idx]
        img_raster_dem = rasterio.open(img_path_dem).read()
        if self.imgs_path==dataset_path_with_labels:
            label = int(self.file_list_dem[idx][-6])
        else:
            label = 'no_label'


        if self.transform:
            img_raster_dem = self.transform(torch.tensor(img_raster_dem))
        
        
        return [img_raster_dem, label]
    
    
    
    
    
def normalize_input_for_dem(test_iter):
    #test_iter = torch.from_numpy(test_iter)
    input_images = test_iter.float()
    #maxv = 822
    #minv = -37
    batch_size = input_images.shape[0]
    cmin = torch.amin(input_images,(1,2)).reshape((batch_size,1,1))
    cmax = torch.amax(input_images,(1,2)).reshape((batch_size,1,1))
    return (input_images-cmin)/(cmax-cmin) #(input_images-minv)/(maxv-minv) #(input_images-cmin)/(cmax-cmin)

class Normalize_range01:
    #bring to range 0 to 1

    def __init__(self, p=1):
        self.p = 1

    def __call__(self, x):
        result = normalize_input_for_dem(x)
        return result

    def __repr__(self):
        return "custom augmentation"
    
class ContrastiveTransformations(object):

    def __init__(self, base_transforms, n_views=2):
        self.base_transforms = base_transforms
        self.n_views = n_views

    def __call__(self, x):
        return [self.base_transforms(x) for i in range(self.n_views)]
    
    


contrast_transforms = transforms.Compose([#transforms.ToTensor(),
                                          #transforms.Resize(size = img_size),
                                          #Normalize_range01(),
                                          #transforms.ToPILImage(), 
                                          #transforms.RandomHorizontalFlip(),
                                          transforms.RandomResizedCrop(size=32),
                                          #transforms.RandomApply([transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.1)], p=0.8),
                                          #transforms.RandomGrayscale(p=0.2),
                                          #transforms.GaussianBlur(kernel_size=9),
                                          #transforms.ToTensor(),
                                          transforms.Normalize((0.5,), (0.5,))
                                         ])

from torch.utils.data import random_split
import math
dataset = SwedishDataset(transform = ContrastiveTransformations(contrast_transforms, n_views = 2), dataset_path = dataset_path)

print(len(dataset))
dataset_len =len(dataset)

train_dataset = torch.utils.data.Subset(dataset, train_set_indexes)
test_dataset = torch.utils.data.Subset(dataset, test_set_indexes)
print(len(train_dataset))
print(len(test_dataset))

#print(dataset[4][0][0].shape)
#dataset[4][1].shape


100652


NameError: name 'train_set_indexes' is not defined

In [4]:
dataset[0][0][0].shape

/home/azamat.kaibaldiyev/miniconda3/envs/contrastive_env/lib/python3.10/site-packages/rasterio/__init__.py:220: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
/home/azamat.kaibaldiyev/miniconda3/envs/contrastive_env/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.tr

torch.Size([1, 32, 32])

In [5]:
dataset[0][0][0]

tensor([[[29.4740, 29.4760, 29.5720,  ..., 28.0480, 27.8200, 27.7920],
         [29.5139, 29.5956, 29.6173,  ..., 28.4214, 28.2894, 27.9479],
         [29.5821, 29.6704, 29.7064,  ..., 28.5123, 28.2840, 28.2121],
         ...,
         [31.1225, 31.2100, 31.2837,  ..., 33.1899, 33.2636, 33.3464],
         [31.2001, 31.2231, 31.3098,  ..., 33.2231, 33.2759, 33.3708],
         [31.2980, 31.3500, 31.3660,  ..., 33.2340, 33.3140, 33.3780]]])

# Model

In [6]:
class SimCLR(pl.LightningModule):

    def __init__(self, hidden_dim, lr, temperature, weight_decay, max_epochs=500):
        super().__init__()
        self.save_hyperparameters()
        assert self.hparams.temperature > 0.0, 'The temperature must be a positive float!'
        # Base model f(.)
        self.convnet = torchvision.models.resnet18(num_classes=4*hidden_dim)  # Output of last linear layer
        #model = torchvision.models.resnet18()
        num_input_channel = 1
        self.convnet.conv1 = nn.Conv2d(num_input_channel, 64, kernel_size=7, stride=2, padding=3,bias=False)
        # The MLP for g(.) consists of Linear->ReLU->Linear
        self.convnet.fc = nn.Sequential(
            self.convnet.fc,  # Linear(ResNet output, 4*hidden_dim)
            nn.ReLU(inplace=True),
            nn.Linear(4*hidden_dim, hidden_dim)
        )

    def configure_optimizers(self):
        optimizer = optim.AdamW(self.parameters(),
                                lr=self.hparams.lr,
                                weight_decay=self.hparams.weight_decay)
        lr_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer,
                                                            T_max=self.hparams.max_epochs,
                                                            eta_min=self.hparams.lr/50)
        return [optimizer], [lr_scheduler]

    def info_nce_loss(self, batch, mode='train'):
        imgs, _ = batch
        imgs = torch.cat(imgs, dim=0)

        # Encode all images
        feats = self.convnet(imgs)
        # Calculate cosine similarity
        cos_sim = F.cosine_similarity(feats[:,None,:], feats[None,:,:], dim=-1)
        # Mask out cosine similarity to itself
        self_mask = torch.eye(cos_sim.shape[0], dtype=torch.bool, device=cos_sim.device)
        cos_sim.masked_fill_(self_mask, -9e15)
        # Find positive example -> batch_size//2 away from the original example
        pos_mask = self_mask.roll(shifts=cos_sim.shape[0]//2, dims=0)
        # InfoNCE loss
        cos_sim = cos_sim / self.hparams.temperature
        nll = -cos_sim[pos_mask] + torch.logsumexp(cos_sim, dim=-1)
        nll = nll.mean()

        # Logging loss
        self.log(mode+'_loss', nll)
        wandb.log({'SimCLR_'+mode+'_loss':nll})
        # Get ranking position of positive example
        comb_sim = torch.cat([cos_sim[pos_mask][:,None],  # First position positive example
                              cos_sim.masked_fill(pos_mask, -9e15)],
                             dim=-1)
        sim_argsort = comb_sim.argsort(dim=-1, descending=True).argmin(dim=-1)
        # Logging ranking metrics
        wandb.log({'SimCLR_'+mode+'_acc_top1':(sim_argsort == 0).float().mean()})
        wandb.log({'SimCLR_'+mode+'_acc_top5':(sim_argsort < 5).float().mean()})
        wandb.log({'SimCLR_'+mode+'_acc_mean_pos': 1+sim_argsort.float().mean()})
        
        self.log(mode+'_acc_top1', (sim_argsort == 0).float().mean())
        self.log(mode+'_acc_top5', (sim_argsort < 5).float().mean())
        self.log(mode+'_acc_mean_pos', 1+sim_argsort.float().mean())

        return nll

    def training_step(self, batch, batch_idx):
        return self.info_nce_loss(batch, mode='train')

    def validation_step(self, batch, batch_idx):
        self.info_nce_loss(batch, mode='val')
        
    def forward(self, x):
        return self.convnet(x)
        
    

In [10]:
def train_simclr(batch_size, max_epochs=500, **kwargs):
    trainer = pl.Trainer(log_every_n_steps=10, default_root_dir=os.path.join(CHECKPOINT_PATH, 'SimCLR'),
                         accelerator="gpu" if str(device).startswith("cuda") else "cpu",
                         devices=1,
                         max_epochs=max_epochs,
                         callbacks=[ModelCheckpoint(save_weights_only=True, mode='max', monitor='val_acc_top5'),
                                    LearningRateMonitor('epoch')])
    trainer.logger._default_hp_metric = None # Optional logging argument that we don't need

    # Check whether pretrained model exists. If yes, load it and skip training
    pretrained_filename = os.path.join(CHECKPOINT_PATH, 'SimCLR.ckpt')
    if os.path.isfile(pretrained_filename):
        print(f'Found pretrained model at {pretrained_filename}, loading...')
        model = SimCLR.load_from_checkpoint(pretrained_filename) # Automatically loads the model with the saved hyperparameters
    else:
        train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                                       drop_last=True, pin_memory=True, num_workers=2)
        val_loader = data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                                     drop_last=False, pin_memory=True, num_workers=2)
        pl.seed_everything(42) # To be reproducable
        model = SimCLR(max_epochs=max_epochs, **kwargs)
        trainer.fit(model, train_loader, val_loader)
        model = SimCLR.load_from_checkpoint(trainer.checkpoint_callback.best_model_path) # Load best checkpoint after training

    return model

# Downstream Task

In [6]:
!wandb login --relogin "40706c6c21e200af3bdb3840883f22e665e74441"

wandb: Appending key for api.wandb.ai to your netrc file: /home/azamat.kaibaldiyev/.netrc


In [52]:
import wandb
import random
from sklearn.metrics import classification_report

epochs = 300
lr = 1e-5
#wandb.init(settings=wandb.Settings(start_method="fork"))
# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="Contrastive_SimCLR_Ohara",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": lr,
    "architecture": "Downstream_Logreg",
    "dataset": "Ohara_dataset_32",
    "epochs": epochs,
    }
)

Downstream_test_acc,▅▅█▆▃▄▃▅█▅▄▃▂▆▅▄▅▃▅▆▅▄▆▃▅▃▆▅▅▃▅▃▃▃▄▃▄▁▄▂
Downstream_test_learningrate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Downstream_test_loss,▃▄▁▂▅▅▆▅▃▄▅▄▇▃▄▃▄▆▄▄▄▆▄▇▄▇▄▅▅▇▄▅█▇▆▇▅▇▆▇
Downstream_train_acc,▁▆▄▅▅▅▆▆▅▆▅▇▆▅▅█▅▇▆▇▆▆▇▆▄▅▇▅▅▇▇▇▆▇▇▇▇▆▆▅
Downstream_train_learningrate,████████████████████████████████████▁▁▁▁
Downstream_train_loss,█▅▄▃▃▃▃▃▃▂▃▃▂▃▃▁▂▂▂▂▂▂▂▂▄▂▁▃▃▁▁▁▂▁▁▂▁▂▂▃
Downstream_val_acc,▂▇▅▂▇▂▅▁▇█▅▅▆▁▇▁▇█▅▅▆▁▅▁▇▇▅▅▆▃▅▁▇▇▅▅▅▃▅▃
Downstream_val_learningrate,██████████████████████████████████▁▁▁▁▁▁
Downstream_val_loss,█▇▆▅▂▆▃▆▂▃▄▄▁▅▂▅▂▂▄▃▁▅▅▄▁▁▃▃▁▅▄▄▁▁▃▃▂▅▄▄
Downstream_test_acc,0.80435
Downstream_test_learningrate,0.0


In [53]:
class LogisticRegression(pl.LightningModule):

    def __init__(self, feature_dim, num_classes, lr, weight_decay, max_epochs=300):
        super().__init__()
        self.save_hyperparameters()
        # Mapping from representation h to classes
        #self.model = nn.Linear(feature_dim, num_classes)
        self.model = nn.Sequential(
                        nn.Linear(feature_dim, num_classes),
                      )
        

    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
        optimizer = optim.AdamW(self.parameters(),
                                lr=self.hparams.lr,
                                weight_decay=self.hparams.weight_decay)
        lr_scheduler = optim.lr_scheduler.MultiStepLR(optimizer,
                                                      milestones=[int(self.hparams.max_epochs*0.6),
                                                                  int(self.hparams.max_epochs*0.8)],
                                                      gamma=0.1)
        return [optimizer], [lr_scheduler]

    def _calculate_loss(self, batch, mode='train'):
        #criterion = F.cross_entropy()
        
        feats, labels = batch
        preds = self.model(feats)
        #print('preds', preds)
        #print('labels:', labels)
        loss = F.cross_entropy(preds, labels)
        acc = (preds.argmax(dim=-1) == labels).float().mean()
        wandb.log({'Downstream_' + mode+'_loss': loss})
        wandb.log({'Downstream_' + mode+'_acc': acc})
        
        lightning_optimizer = self.optimizers()  # self = your model
        current_lr = lightning_optimizer.optimizer.param_groups[0]['lr']
        wandb.log({'Downstream_' + mode+'_learningrate': current_lr})

        self.log(mode + '_loss', loss)
        self.log(mode + '_acc', acc)
        return loss

    def training_step(self, batch, batch_idx):
        return self._calculate_loss(batch, mode='train')

    def validation_step(self, batch, batch_idx):
        self._calculate_loss(batch, mode='val')

    def test_step(self, batch, batch_idx):
        self._calculate_loss(batch, mode='test')

In [54]:

with open('files_remove_indices/Ohara_labels_train_set_indexes.npy', 'rb') as f:
    train_set_indexes_labels = np.load(f)

with open('files_remove_indices/Ohara_labels_test_set_indexes.npy', 'rb') as f:
    test_set_indexes_labels = np.load(f)

In [55]:
img_transforms = transforms.Compose([#Normalize_range01(),
                                     transforms.Normalize((0.5,), (0.5,))])

#dataset_path_with_labels ='Tasmania_ohara_labels/cropped_dems_16_with_labels_correct/'
dataset_path_with_labels = 'Tasmania_ohara_labels/cropped_dems_32_with_labels/'

dataset = SwedishDataset(transform = img_transforms , dataset_path=dataset_path_with_labels)
# #For random split
train_dataset = torch.utils.data.Subset(dataset, train_set_indexes_labels)
test_dataset = torch.utils.data.Subset(dataset, test_set_indexes_labels)

#For different lines
# dim = 4460
# train_dataset = torch.utils.data.Subset(dataset, np.arange(dim))
# test_dataset = torch.utils.data.Subset(dataset, np.arange(dim, len(dataset)))

print(len(dataset))
print("Number of training examples:", len(train_dataset))
print("Number of test examples:", len(test_dataset))

10967
Number of training examples: 9321
Number of test examples: 1646


In [45]:
@torch.no_grad()
def prepare_data_features(model, dataset):
    # Prepare model
    network = deepcopy(model.convnet)
    network.fc = nn.Identity()  # Removing projection head g(.)
    network.eval()
    network.to(device)

    # Encode all images
    data_loader = data.DataLoader(dataset, batch_size=32, num_workers=NUM_WORKERS, shuffle=False, drop_last=False)
    feats, labels = [], []
    for batch_imgs, batch_labels in tqdm(data_loader):
        batch_imgs = batch_imgs.to(device)
        batch_feats = network(batch_imgs)
        feats.append(batch_feats.detach().cpu())
        labels.append(batch_labels)

    feats = torch.cat(feats, dim=0)
    labels = torch.cat(labels, dim=0)

    # Sort images by labels
    #labels, idxs = labels.sort()
    #feats = feats[idxs]
    #mycode
    feats = torch.squeeze(feats)
    labels = torch.squeeze(labels)

    return data.TensorDataset(feats, labels)

In [56]:
class SimCLR(pl.LightningModule):

    def __init__(self, hidden_dim, lr, temperature, weight_decay, max_epochs=500):
        super().__init__()
        self.save_hyperparameters()
        assert self.hparams.temperature > 0.0, 'The temperature must be a positive float!'
        # Base model f(.)
        self.convnet = torchvision.models.resnet18(num_classes=4*hidden_dim)  # Output of last linear layer
        #model = torchvision.models.resnet18()
        num_input_channel = 1
        self.convnet.conv1 = nn.Conv2d(num_input_channel, 64, kernel_size=7, stride=2, padding=3,bias=False)
        # The MLP for g(.) consists of Linear->ReLU->Linear
        self.convnet.fc = nn.Sequential(
            self.convnet.fc,  # Linear(ResNet output, 4*hidden_dim)
            nn.ReLU(inplace=True),
            nn.Linear(4*hidden_dim, hidden_dim)
        )

    def configure_optimizers(self):
        optimizer = optim.AdamW(self.parameters(),
                                lr=self.hparams.lr,
                                weight_decay=self.hparams.weight_decay)
        lr_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer,
                                                            T_max=self.hparams.max_epochs,
                                                            eta_min=self.hparams.lr/50)
        return [optimizer], [lr_scheduler]

    def info_nce_loss(self, batch, mode='train'):
        imgs, _ = batch
        imgs = torch.cat(imgs, dim=0)

        # Encode all images
        feats = self.convnet(imgs)
        # Calculate cosine similarity
        cos_sim = F.cosine_similarity(feats[:,None,:], feats[None,:,:], dim=-1)
        # Mask out cosine similarity to itself
        self_mask = torch.eye(cos_sim.shape[0], dtype=torch.bool, device=cos_sim.device)
        cos_sim.masked_fill_(self_mask, -9e15)
        # Find positive example -> batch_size//2 away from the original example
        pos_mask = self_mask.roll(shifts=cos_sim.shape[0]//2, dims=0)
        # InfoNCE loss
        cos_sim = cos_sim / self.hparams.temperature
        nll = -cos_sim[pos_mask] + torch.logsumexp(cos_sim, dim=-1)
        nll = nll.mean()

        # Logging loss
        self.log(mode+'_loss', nll)
        # Get ranking position of positive example
        comb_sim = torch.cat([cos_sim[pos_mask][:,None],  # First position positive example
                              cos_sim.masked_fill(pos_mask, -9e15)],
                             dim=-1)
        sim_argsort = comb_sim.argsort(dim=-1, descending=True).argmin(dim=-1)
        # Logging ranking metrics
     
        self.log(mode+'_acc_top1', (sim_argsort == 0).float().mean())
        self.log(mode+'_acc_top5', (sim_argsort < 5).float().mean())
        self.log(mode+'_acc_mean_pos', 1+sim_argsort.float().mean())

        return nll

    def training_step(self, batch, batch_idx):
        return self.info_nce_loss(batch, mode='train')

    def validation_step(self, batch, batch_idx):
        self.info_nce_loss(batch, mode='val')
        
    def forward(self, x):
        return self.convnet(x)
        
    

In [7]:
!ls Tasmania_ohara_labels/checkpoints_dems_16/SimCLR/lightning_logs/version_1344416/checkpoints/


'epoch=466-step=164851.ckpt'


In [27]:
!ls Tasmania_ohara_labels/checkpoints_dems_32/SimCLR/lightning_logs/version_1344418/checkpoints/

'epoch=479-step=169440.ckpt'


In [57]:
best_checkpoint = '/home/azamat.kaibaldiyev/Tasmania_ohara_labels/checkpoints_dems_32/SimCLR/lightning_logs/version_1344418/checkpoints/epoch=479-step=169440.ckpt'

simclr_model = SimCLR.load_from_checkpoint(best_checkpoint)

# disable randomness, dropout, etc...
simclr_model.eval()
simclr_model.to(device)

SimCLR(
  (convnet): ResNet(
    (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runn

In [48]:
train_feats_simclr = prepare_data_features(simclr_model, train_dataset)
test_feats_simclr = prepare_data_features(simclr_model, test_dataset)

  0%|          | 0/292 [00:00<?, ?it/s]

  0%|          | 0/52 [00:00<?, ?it/s]

In [58]:
def train_logreg(batch_size, train_feats_data, test_feats_data, model_suffix, max_epochs=200, **kwargs):
    trainer = pl.Trainer(log_every_n_steps=10, default_root_dir=os.path.join(CHECKPOINT_PATH, "LogisticRegression"),
                         accelerator="gpu" if str(device).startswith("cuda") else "cpu",
                         devices=1,
                         max_epochs=max_epochs,
                         callbacks=[ModelCheckpoint(save_weights_only=True, mode='max', monitor='val_acc'),
                                    LearningRateMonitor("epoch")],
                         enable_progress_bar=False,
                         check_val_every_n_epoch=10)
    trainer.logger._default_hp_metric = None

    # Data loaders
    train_loader = data.DataLoader(train_feats_data, batch_size=batch_size, shuffle=True,
                                   drop_last=False, pin_memory=True, num_workers=0)
    test_loader = data.DataLoader(test_feats_data, batch_size=batch_size, shuffle=False,
                                  drop_last=False, pin_memory=True, num_workers=0)

    # Check whether pretrained model exists. If yes, load it and skip training
    pretrained_filename = os.path.join(CHECKPOINT_PATH, f"LogisticRegression_{model_suffix}.ckpt")
    if os.path.isfile(pretrained_filename):
        print(f"Found pretrained model at {pretrained_filename}, loading...")
        model = LogisticRegression.load_from_checkpoint(pretrained_filename)
    else:
        pl.seed_everything(42)  # To be reproducable
        model = LogisticRegression(**kwargs)
        trainer.fit(model, train_loader, test_loader)
        model = LogisticRegression.load_from_checkpoint(trainer.checkpoint_callback.best_model_path)

    # Test best model on train and validation set
    train_result = trainer.test(model, train_loader, verbose=False)
    test_result = trainer.test(model, test_loader, verbose=False)
    result = {"train": train_result[0]["test_acc"], "test": test_result[0]["test_acc"]}

    return model, result

In [59]:
num_imgs_per_label = 200
logreg_model, small_set_results = train_logreg(max_epochs=200,batch_size=128,
                                    train_feats_data=train_feats_simclr,
                                    test_feats_data=test_feats_simclr,
                                    model_suffix=num_imgs_per_label,
                                    feature_dim=train_feats_simclr.tensors[0].shape[1],
                                    num_classes=8,
                                    lr=1e-5,
                                    weight_decay=1e-3,)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[rank: 0] Global seed set to 42
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 4.1 K 
-------------------------------------
4.1 K     Trainable params
0         Non-trainable params
4.1 K     Total params
0.016     Total estimated model params size (MB)
`Trainer.fit` stopped: `max_epochs=200` reached.
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly

In [18]:
#size 16, bs 64, 200 epochs, line set
small_set_results

{'train': 0.7721973061561584, 'test': 0.6397725343704224}

In [38]:
#size 32, bs 64, 200 epochs, line set
small_set_results

{'train': 0.652242124080658, 'test': 0.5979714393615723}

In [25]:
#size 16, bs 128, 300 epochs, random set
small_set_results

{'train': 0.7273897528648376, 'test': 0.7290400862693787}

In [19]:
#size 16, bs 64, 300 epochs, random set
small_set_results

{'train': 0.7292135953903198, 'test': 0.7320777773857117}

In [51]:
#size 32, bs 64, 200 epochs, random set
small_set_results

{'train': 0.7647247910499573, 'test': 0.763669490814209}

In [69]:
#size 32, bs 128, 100 epochs, random set
small_set_results

{'train': 0.7595751285552979, 'test': 0.7527338862419128}

In [60]:
#size 32, bs 128, 100 epochs, random set   #2 (latest)
small_set_results

{'train': 0.764832079410553, 'test': 0.7618468999862671}